# Load Models and Tokenizers

In [1]:
from tensorflow.python.keras.models import load_model

C:\Users\tejks\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Encoder and decoder model

In [2]:
encoder_model = load_model('seq2seq_encoder_eng_hin.hd5')

In [3]:
decoder_model = load_model('seq2seq_decoder_eng_hin.hd5')

Load tokenizers

In [4]:
import pickle

In [5]:
encoder_t = pickle.load(open('encoder_tokenizer_eng','rb'))

In [6]:
decoder_t = pickle.load(open('decoder_tokenizer_hin','rb'))

Define Configuration parameters

In [7]:
max_encoder_seq_length = 22 #From the training

In [8]:
max_decoder_seq_length = 27 #From the training

In [9]:
#Dictionary to convert Decoder output into Hindi Words 
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

# Function to generate Padded sequences for Input string

In [10]:
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [11]:
def encode_input(input_str):
    
    #Convert words to indexes
    encoder_seq = encoder_t.texts_to_sequences([input_str])
    
    #Pad sequences
    encoder_input_data = pad_sequences(encoder_seq, maxlen=max_encoder_seq_length, padding='post')
    
    return encoder_input_data    

# Prediction Function

In [12]:
import numpy as np

In [13]:
def decode_sentence(input_str):
    
    #Convert input string to padded sequence
    input_seq = encode_input(input_str)
    
    #Get the encoder state values
    decoder_initial_states_value = encoder_model.predict(input_seq)
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    #start the loop
    while not stop_loop:
        
        predicted_outputs, h, c = decoder_model.predict([target_seq] + 
                                                        decoder_initial_states_value)
        
        #Get the predicted output with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter integer
        predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or len(predicted_sentence) > max_decoder_seq_length):
            
            stop_loop = True
            continue
                    
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
    
    return predicted_sentence

In [14]:
if __name__== '__main__':
    print(decode_sentence('How are you?'))

आप कैसे हैं
